Завдання 3. Реалізувати одну або декілька задач (за вибором):

3. Синтезатор. Розробити елементи власного синтезатора музикального інструменту.

In [16]:
import numpy as np
import pyaudio as pa
import pygame

In [17]:
RATE = 44100
DURATION = 1/2
AMPLITUDE = 2 ** 16

n = np.array([-9, -7, -5, -4, -2, 0, 2])
frequencies = 440 * np.power(2 ** (1/12), n)
frequencies

array([261.6255653 , 293.66476792, 329.62755691, 349.22823143,
       391.99543598, 440.        , 493.88330126])

In [18]:
pygame.init()
pygame.mixer.init()

sounds = []
for freq in frequencies:
    note = np.sin(2 * np.pi * freq * np.linspace(0, DURATION, round(DURATION * RATE)))
    # sound = np.array(note, dtype=np.int16)
    # sound = np.asarray([32767*note, 32767*note]).T.astype(np.int16)
    # sound = pygame.sndarray.make_sound(sound.copy())
    sounds.append(note)

In [19]:
sounds[0]

array([ 0.        ,  0.03726838,  0.07448499, ..., -0.94926027,
       -0.93688022, -0.92319845])

In [20]:
keys = 'asdfghjkl'[:len(sounds)]
ord_list = [ord(key) for key in keys]
key_pressed = {key:False for key in ord_list}
key_sound = {ord_list[i]:sounds[i] for i in range(len(sounds))}

p = pa.PyAudio()
stream = p.open(format=pa.paInt16,
                channels=2, rate=RATE, output=True)
window_size = 320, 240
screen = pygame.display.set_mode(window_size)
pygame.display.flip()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.KEYDOWN:
            if event.key == ord('q'):
                running = False

            if event.key in key_pressed:
                key_pressed[event.key] = True
            
        if event.type == pygame.KEYUP:
            if event.key in key_pressed:
                key_pressed[event.key] = False

    mixed_sound = np.zeros(round(DURATION * RATE))
    count = 0
    for key in key_pressed:
        if key_pressed[key] == True:
            # stream.write(key_sound[key])
            count += 1
            mixed_sound += key_sound[key]
    mixed_sound /= count
    sound = np.asarray([32767*mixed_sound, 32767*mixed_sound]).T.astype(np.int16)
    pygame.sndarray.make_sound(sound.copy()).play()


pygame.quit()
stream.stop_stream()
stream.close()
p.terminate()

/var/folders/7l/ngvfjxqd08lbrvzywpzh7pfr0000gn/T/ipykernel_21272/1959979102.py:36: RuntimeWarning: invalid value encountered in divide
  mixed_sound /= count
